In [1]:
# #Setup Gdrive file download extention
!pip install open-clip-torch faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.6 MB/s eta 0:00:00


In [2]:
# !gdown --id 1BFAJfzzeaUGsPoYELS86HIutJ43D-vat

In [3]:
import os
import math

import numpy as np

import torch as th
import torch.nn as nn
import torch.nn.functional as F
import open_clip

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision.transforms as T

from tqdm import tqdm
import pandas as pd
import cv2
from PIL import Image
import time
import faiss
import copy

In [4]:
# import torchvision.models as models

# # List some of the available pretrained models
# pretrained_models = dir(models)
# print(pretrained_models)

In [5]:
import argparse

import numpy as np
import pandas as pd

from tqdm import tqdm


def compute_precision_at_k(ranked_targets: np.ndarray,
                           k: int) -> float:

    """
    Computes the precision at k.
    Args:
        ranked_targets: A boolean array of retrieved targets, True if relevant and False otherwise.
        k: The number of examples to consider

    Returns: The precision at k
    """
    assert k >= 1
    assert ranked_targets.size >= k, ValueError('Relevance score length < k')
    return np.mean(ranked_targets[:k])

def compute_average_precision(ranked_targets: np.ndarray,
                              gtp: int) -> float:
    
        
    """
    Computes the average precision.
    Args:
        ranked_targets: A boolean array of retrieved targets, True if relevant and False otherwise.
        gtp: ground truth positives.

    Returns:
        The average precision.
    """
    assert gtp >= 1
    # compute precision at rank only for positive targets
    out = [compute_precision_at_k(ranked_targets, k + 1) for k in range(ranked_targets.size) if ranked_targets[k]]
    if len(out) == 0:
        # no relevant targets in top1000 results
        return 0.0
    else:
        return np.sum(out) / gtp


def calculate_map(ranked_retrieval_results: np.ndarray,
                  query_labels: np.ndarray,
                  gallery_labels: np.ndarray) -> float:
    
    global current_retrievals, gpt
    
    """
    Calculates the mean average precision.
    Args:
        ranked_retrieval_results: A 2D array of ranked retrieval results (shape: n_queries x 1000), because we use
                                top1000 retrieval results.
        query_labels: A 1D array of query class labels (shape: n_queries).
        gallery_labels: A 1D array of gallery class labels (shape: n_gallery_items).
    Returns:
        The mean average precision.
    """
    assert ranked_retrieval_results.ndim == 2
    assert ranked_retrieval_results.shape[1] == 1000

    class_average_precisions = []
    current_retrievals = []

    class_ids, class_counts = np.unique(gallery_labels, return_counts=True)
    class_id2quantity_dict = dict(zip(class_ids, class_counts))
    for gallery_indices, query_class_id in tqdm(
                            zip(ranked_retrieval_results, query_labels),
                            total=len(query_labels)):
        # Checking that no image is repeated in the retrival results
        assert len(np.unique(gallery_indices)) == len(gallery_indices), \
                    ValueError('Repeated images in retrieval results')

        current_retrieval = gallery_labels[gallery_indices] == query_class_id
        gpt = class_id2quantity_dict[query_class_id]
        
        current_retrievals.append(current_retrieval)

        class_average_precisions.append(
            compute_average_precision(current_retrieval, gpt)
        )

    mean_average_precision = np.mean(class_average_precisions)
    return mean_average_precision

In [6]:
import numpy as np
import time

def calculate_l2_distances(query, gallery):
    return np.linalg.norm(gallery - query, axis=1)

def get_k_nearest_neighbors(distances, k):
    indices = np.argsort(distances)[:k]
    return indices

def get_similiarity_l2(embeddings_gallery, embeddings_query, k):
    print('Processing indices...')

    s = time.time()

    scores = []
    indices = []

    for query in embeddings_query:
        distances = calculate_l2_distances(query, embeddings_gallery)
        nearest_indices = get_k_nearest_neighbors(distances, k)
        scores.append(distances[nearest_indices])
        indices.append(nearest_indices)

    e = time.time()

    print(f'Finished processing indices, took {e - s}s')
    return np.array(scores), np.array(indices)

In [7]:
def convert_indices_to_labels(indices, labels):
    indices_copy = copy.deepcopy(indices)
    for row in indices_copy:
        for j in range(len(row)):
            row[j] = labels[row[j]]
    return indices_copy

In [8]:
device = 'cuda' if th.cuda.is_available() else 'cpu';

# Reading Dataset

In [9]:
def read_image(image_file):
    img = cv2.imread(
        image_file, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION
    )
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if img is None:
        raise ValueError('Failed to read {}'.format(image_file))
    return img


class SubmissionDataset(Dataset):
    def __init__(self, root, annotation_file, transforms, with_bbox=False):
        self.root = root
        self.imlist = pd.read_csv(annotation_file)
        self.transforms = transforms
        self.with_bbox = with_bbox

    def __getitem__(self, index):
        cv2.setNumThreads(6)

        full_imname = os.path.join(self.root, self.imlist['img_path'][index])
        img = read_image(full_imname)

        if self.with_bbox:
            x, y, w, h = self.imlist.loc[index, 'bbox_x':'bbox_h']
            img = img[y:y+h, x:x+w, :]

        img = Image.fromarray(img)
        img = self.transforms(img)
        product_id = self.imlist['product_id'][index]
        return img, product_id

    def __len__(self):
        return len(self.imlist)

In [10]:
def get_transform():  
    transform = T.Compose([
            T.Resize(
                size=(224, 224), 
                interpolation=T.InterpolationMode.BICUBIC,
                antialias=True),
            T.ToTensor(), 
            T.Normalize(
                mean=(0.48145466, 0.4578275, 0.40821073), 
                std=(0.26862954, 0.26130258, 0.27577711)
            )
        ])
    return transform

@th.no_grad()
def get_feature_vector(model, dataloader, use_cuda=True):
    features = []
    product_id = []
    
    for imgs, p_id in tqdm(dataloader):
        if use_cuda:
            imgs = imgs.cuda()
        features.append(th.squeeze(model(imgs.half())).detach().cpu().numpy().astype(np.float32))
        product_id.append(th.squeeze(p_id).detach().cpu().numpy())

    return np.concatenate(features, axis=0), np.concatenate(product_id)

# Getting the CLIP model's embedding

In [11]:
weights_path = '/kaggle/input/vit-h-14-final-weights/model_weights.pt'

In [12]:
vit_backbone = open_clip.create_model_and_transforms('ViT-H-14', None)[0].visual
vit_backbone.load_state_dict(th.load(weights_path))
vit_backbone.half()   # Apply half precision to the backbone model
vit_backbone.eval()   # Dropping unecessary layers
model = vit_backbone
model.cuda()

VisionTransformer(
  (patchnorm_pre_ln): Identity()
  (conv1): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1280, out_features=1280, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
)

In [13]:
# import torch
# # Specify the path for the binary file
# binary_file_path = "/kaggle/working/model.bin"

# # Save the model's state dictionary to the binary file
# torch.save(model, binary_file_path)

In [14]:
# import zipfile
# import os

# def zip_folder(folder_path, zip_filename):
#     with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, _, files in os.walk(folder_path):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 arcname = os.path.relpath(file_path, folder_path)
#                 zipf.write(file_path, arcname)

# # Replace 'your_folder_path' with the actual path to the folder you want to zip
# folder_to_zip = '/kaggle/working'
# output_zip_path = 'VIT-H-14.zip'

# zip_folder(folder_to_zip, output_zip_path)

# from IPython.display import FileLink

# # Display a download link for the zip file
# FileLink(output_zip_path)

In [15]:
# !rm model1.pt

In [16]:
transform = get_transform()

img_dir = "/kaggle/input/vprtestdata/public_dataset/"

dataset_train = SubmissionDataset(img_dir, os.path.join(img_dir, "gallery.csv"), transform)
dataloader_train = DataLoader(dataset_train, batch_size=512, num_workers=4)
dataset_test = SubmissionDataset(img_dir, os.path.join(img_dir, "queries.csv"), transform, with_bbox=True)
dataloader_test = DataLoader(dataset_test, batch_size=512, num_workers=4)

In [17]:
# def predict(model):
#     global feature_vectors_gallery, labels_gallery
    
#     feature_vectors_gallery, labels_gallery = get_feature_vector(model, dataloader_train, 1)
#     feature_vectors_query, labels_query = get_feature_vector(model, dataloader_test, 1)
    
#     # Get the 5 best matching product indices
#     scores, indices = get_similiarity_l2(feature_vectors_gallery, feature_vectors_query, 5)
#     indices = indices.tolist()
# #     print('indices', len(indices))
#     labels_gallery = labels_gallery.tolist()
#     labels_query = labels_query.tolist()
    
#     # Define the weights for the weighted average
#     weights = [0.65, 0.3 , 0.1, 0.1, 0.1, 0.1]

#     # Compute the weighted average of the query and the 5 matches embeddings
    
#     features = []
#     for i, feature_vector in enumerate(feature_vectors_query):
# #         print('feature vector', feature_vector.shape)
#         weighted_average_query = feature_vector * weights[0]
# #         print('weighted_average_query vector', weighted_average_query.shape)
#         indices_query = indices[i]
# #         print(indices)
#         for i, idx in enumerate(indices_query):
#             weighted_average_query += feature_vectors_gallery[idx] * weights[i + 1]
#         features.append(weighted_average_query)
# #     print(len(features))

#     # Calculate the similarity between the new query and all gallery embeddings
#     scores, indices = get_similiarity_l2(feature_vectors_gallery, features, 1000)
#     indices = indices.tolist()

#     return indices

In [18]:
weights = [
[0.4, 0.3 , 0.2, 0.1, 0, 0],
[0.4, 0.35 , 0.25, 0.1, 0, 0],
[0.4, 0.35 , 0.2, 0.05, 0, 0],
[0.4, 0.3 , 0.25, 0.05, 0, 0],
[0.4, 0.4 , 0.2, 0, 0, 0],
[0.45, 0.25 , 0.2, 0.2, 0, 0],
[0.45, 0.3 , 0.2, 0.1, 0, 0],
[0.45, 0.35 , 0.2, 0, 0, 0],
[0.45, 0.25 , 0.2, 0.05, 0, 0],
[0.45, 0.3 , 0.2, 0, 0, 0],
[0.5, 0.2 , 0.2, 0.1, 0, 0],
[0.5, 0.25 , 0.1, 0.15, 0, 0],
[0.5, 0.3 , 0.15, 0.05, 0, 0],
[0.5, 0.35 , 0.15, 0, 0, 0],
[0.5, 0.3 , 0.1, 0.1, 0, 0],
[0.5, 0.3 , 0.2, 0.1, 0, 0],
[0.55, 0.15 , 0.05, 0, 0, 0],
[0.55, 0.20 , 0.1, 0, 0, 0],
[0.55, 0.25 , 0.15, 0, 0, 0],
[0.55, 0.3 , 0.2, 0, 0, 0],
[0.55, 0.35 , 0.1, 0, 0, 0],
[0.6, 0.3 , 0.2, 0, 0, 0],
[0.6, 0.35 , 0.2, 0, 0, 0],
[0.6, 0.20 , 0.2, 0, 0, 0],
[0.6, 0.25 , 0.2, 0, 0, 0],
[0.6, 0.15 , 0.2, 0, 0, 0]
]

In [19]:
def predict(model, weight):
    global feature_vectors_gallery, labels_gallery
    
    feature_vectors_gallery, labels_gallery = get_feature_vector(model, dataloader_train, 1)
    feature_vectors_query, labels_query = get_feature_vector(model, dataloader_test, 1)
    
#     feature_vectors_query, feature_vectors_gallery = db_augmentation(feature_vectors_query, feature_vectors_gallery, top_k=10)
#     feature_vectors_query, feature_vectors_gallery = average_query_expansion(feature_vectors_query, feature_vectors_gallery, top_k=5)
    
    scores, indices = get_similiarity_l2(feature_vectors_gallery, feature_vectors_query, 5)

    indices = indices.tolist()
    labels_gallery = labels_gallery.tolist()
    labels_query = labels_query.tolist()
    
        # Define the weights for the weighted average
    weights = weight

    # Compute the weighted average of the query and the 5 matches embeddings
    
    features = []
    for i, feature_vector in enumerate(feature_vectors_query):
#         print('feature vector', feature_vector.shape)
        weighted_average_query = feature_vector * weights[0]
#         print('weighted_average_query vector', weighted_average_query.shape)
        indices_query = indices[i]
#         print(indices)
        for i, idx in enumerate(indices_query):
            weighted_average_query += feature_vectors_gallery[idx] * weights[i + 1]
        features.append(weighted_average_query)
#     print(len(features))

    # Calculate the similarity between the new query and all gallery embeddings
    scores, indices = get_similiarity_l2(feature_vectors_gallery, features, 1000)
    indices = indices.tolist()

    return indices

In [20]:
# preds = predict(model)

# preds_df = pd.DataFrame(preds)
# preds_df.to_csv('preds.csv', index=False)

In [21]:
# preds

In [22]:
# preds_df.head()

In [23]:
seller_gt = pd.read_csv('/kaggle/input/vprtestdata/public_dataset/gallery.csv')
gallery_labels = seller_gt['product_id'].values
user_gt = pd.read_csv('/kaggle/input/vprtestdata/public_dataset/queries.csv')
query_labels = user_gt['product_id'].values

# # Evalaute metrics
# print("Evaluation Results")
# results = {"mAP": calculate_map(np.array(preds), query_labels, gallery_labels)}
# print(results)

In [24]:
for weight in weights:
    preds = predict(model, weight)
    print("Weight", weight)
    results = {"mAP": calculate_map(np.array(preds), query_labels, gallery_labels)}
    print(results)
    print('-------------------------------------------------')

100%|██████████| 4/4 [01:09<00:00, 17.40s/it]


Processing indices...
Finished processing indices, took 2.94952392578125s
Processing indices...
Finished processing indices, took 2.9748902320861816s
Weight [0.4, 0.3, 0.2, 0.1, 0, 0]


100%|██████████| 1935/1935 [00:00<00:00, 6462.73it/s]


{'mAP': 0.6045218741718739}
-------------------------------------------------


100%|██████████| 4/4 [01:08<00:00, 17.08s/it]


...

{'mAP': 0.6055401956309567}
-------------------------------------------------


100%|██████████| 4/4 [01:08<00:00, 17.08s/it]


Processing indices...
Finished processing indices, took 2.8954994678497314s
Processing indices...
Finished processing indices, took 2.919609785079956s
Weight [0.6, 0.15, 0.2, 0, 0, 0]


100%|██████████| 1935/1935 [00:00<00:00, 6402.54it/s]

{'mAP': 0.5933886271791328}
-------------------------------------------------


In [25]:
# normal mAP = {'mAP': 0.6076191768389487}

# db_augmentation = {'mAP': 0.6057361533665884}

# with only query AWQ = {'mAP': 0.6057361533665884}

# weights = [0.4, 0.2, 0.1, 0.1, 0.1, 0.1]
# {'mAP': 0.5953972874661622}

# weights = [0.5, 0.2, 0.1, 0.1, 0.1, 0.1]
# {'mAP': 0.5986392800555753}

# weights = [0.6, 0.2, 0.1, 0.1, 0.1, 0.1]
# {'mAP': 0.6002180796285603}

# weights = [0.7, 0.2, 0.1, 0.1, 0.1, 0.1]
# {'mAP': 0.601189472498224}

# weights = [0.7, 0.3 , 0.1, 0.1, 0.1, 0.1]
# {'mAP': 0.6057971805514993}

# weights = [0.7, 0.3 , 0.2, 0.1, 0.1, 0.1]
# {'mAP': 0.6026055615146477}

# weights = [0.7, 0.4 , 0.1, 0.1, 0.1, 0.1]
# {'mAP': 0.6042976589391011}

# weights = [0.7, 0.35 , 0.1, 0.1, 0.1, 0.1]
# {'mAP': 0.6043665158358905}

# weights = [0.65, 0.3 , 0.1, 0.1, 0.1, 0.1]
# 'mAP': 0.6051452262908571}

# weights = [0.65, 0.3 , 0.1, 0.1, 0, 0]
# {'mAP': 0.6065329259732295}

# weights = [0.65, 0.3 , 0.1, 0, 0, 0]
# {'mAP': 0.6064506837973526}

# weights = [0.65, 0.4 , 0.1, 0, 0, 0]
# {'mAP': 0.6065735465000247